# ALGOS DE RECO #

In [143]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix

# on importe les différentes librairies surprise de scikit
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans

# on importe tout ce qui est en rapport avec le nlp
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# on importe notre bdd
avis = pd.read_csv("BDD/avis.csv", index_col="Unnamed: 0")
avis.drop(['url'], axis=1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hacho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hacho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,author,date_published,title_review,note,title,comment
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,8,Mariposas,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,morlockbob,2020-10,le festival de Michoacan,7,Mariposas,Comment continuer après un mega hit ? Simpleme...
2,SwatSh,2021-02,Vivez la migration des monarques,7,Mariposas,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,Timi JeuxATheme,2020-11,Bon,8,Mariposas,NaN
4,prunelles,2021-05,Envolez-moi,9,Mariposas,"Très joli bijou que ce jeu-là ! Le matériel, l..."
...,...,...,...,...,...,...
176205,Xian9x,2008-11,piou piou,8,Mag Blast,"Mag blast est un jeu rapide, drole et sans pré..."
176206,marreco,2007-11,peewww peeeww t mort,4,Mag Blast,je n'ai pu qu'y jouer à 2 et c'est moyen. On a...
176208,EtienneS,2004-01,1000 bornes dans l'espace,6,Mag Blast,"MagBlast comporte certaines très bonnes idées,..."
176209,nicolasthomasse,2017-11,Bon,7,Mag Blast,NaN


In [144]:
avis.head(1)

,author,date_published,title_review,note,title,url,comment
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po..."


Il faut mettre en place la matrice sparse (calculer la sparsité + plot cette sparsite)

In [145]:
num_users  = avis["author"].nunique()
num_items  = avis["title"].nunique()

print(f"Il y'a {num_users} auteurs et {num_items} jeux")

sparsity = (len(avis) / (num_users*num_items))*100
print(f"Notre matrice a une sparsité de {sparsity:.2f}% ")

matrice_sparse = csr_matrix(avis['note'])
matrice_sparse

Il y'a 2564 auteurs et 3057 jeux
Notre matrice a une sparsité de 1.60% 


<1x125299 sparse matrix of type '<class 'numpy.int64'>'
	with 124925 stored elements in Compressed Sparse Row format>

On test maintenant les differents algos (SVD + KNN pour commencer)

## SVD ##

In [146]:
algo = SVD()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(avis, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8815  1.8628  1.8773  1.8641  1.8659  1.8703  0.0076  
MAE (testset)     1.4456  1.4292  1.4373  1.4288  1.4358  1.4353  0.0062  
Fit time          0.84    0.85    0.87    0.82    0.83    0.84    0.02    
Test time         0.07    0.24    0.08    0.08    0.07    0.11    0.07    


{'test_rmse': array([1.88150924, 1.86280541, 1.87726886, 1.86412786, 1.86589586]),
 'test_mae': array([1.44560688, 1.42917496, 1.43728139, 1.42878654, 1.43578278]),
 'fit_time': (0.8390226364135742,
  0.8468790054321289,
  0.8725428581237793,
  0.8170163631439209,
  0.830923318862915),
 'test_time': (0.0749964714050293,
  0.2446444034576416,
  0.07897019386291504,
  0.07549309730529785,
  0.07400298118591309)}

## k-NN ##

In [147]:
algo = KNNBasic()

reader = Reader(line_format='user item rating', rating_scale=(0,10))

class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, jeux, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(avis['author'], avis['title'], avis['note'])]
        self.reader=reader

data = MyDataset(avis, reader)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9249  1.9164  1.9098  1.9185  1.9080  1.9155  0.0061  
MAE (testset)     1.4886  1.4864  1.4787  1.4867  1.4797  1.4840  0.0040  
Fit time          0.43    0.44    0.44    0.44    0.44    0.44    0.00    
Test time         0.99    0.99    1.14    1.01    0.99    1.02    0.06    


{'test_rmse': array([1.92490483, 1.91635336, 1.90978445, 1.91850063, 1.90804607]),
 'test_mae': array([1.48863644, 1.48635525, 1.47869011, 1.48672127, 1.47966394]),
 'fit_time': (0.4345521926879883,
  0.4400908946990967,
  0.44379281997680664,
  0.4410429000854492,
  0.4424412250518799),
 'test_time': (0.9851007461547852,
  0.9932329654693604,
  1.1392545700073242,
  1.0102150440216064,
  0.9921379089355469)}

In [148]:
avis_par_jeux = avis.groupby('title').size()
avis_par_jeux.to_dict()

avis_trie = dict(sorted(avis_par_jeux.items(), key=lambda item: item[1], reverse=True))

dix_premieres_valeurs = {k: avis_trie[k] for k in list(avis_trie)[:10]}

In [149]:
dix_premieres_valeurs = {k: avis_trie[k] for k in list(avis_trie)[:10]}

In [150]:
dix_premiers_jeux_liste = list(dix_premieres_valeurs)[:10]
df_filtre = avis[avis['title'].isin(dix_premiers_jeux_liste)]
df_filtre.drop(['url','date_published'], axis=1)

,author,title_review,note,title,comment
5096,PtitJu,Une partie à l’occasion : OK. Une seconde part...,6,Non Merci,Les + :\nRègles simples donc jeu idéal pour jo...
5097,morlockbob,avec plaisir ;-),7,Non Merci,retrouver mon avis éclairé sur jairienadiremai...
5098,M.PinK,Bon,8,Non Merci,NaN
5099,beRtuj,Pas mauvais,6,Non Merci,NaN
5100,Nicolas44,Pas mauvais,6,Non Merci,NaN
...,...,...,...,...,...
163487,scand1sk,L'archétype du Risk-like,2,Shogun,"Dans les années 1970 à 1980, en termes de jeux..."
163488,OFeuque,Un Sumo du plateau,8,Shogun,Attention GROS jeu.De la même veine qu'un Twil...
163489,R2,Oui mais...,6,Shogun,J'aime les jeux avec du beau matériel qui débo...
163490,crashphil,Un monstre très jouable,7,Shogun,J'ai acheté cette impressionnante boîte lors d...


In [152]:
def preprocess_comment(comment):

    if isinstance(comment, float):
        return ''
    
    # Tokenisation
    tokens = word_tokenize(comment)
    
    # Suppression des stopwords
    stop_words = set(stopwords.words('french'))
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return tokens

# Appliquer la fonction de nettoyage des commentaires à chaque commentaire
df_filtre['comment_cleaned'] = df_filtre['comment'].apply(preprocess_comment)

# Compter la fréquence des mots dans tous les commentaires
word_freq = Counter()
for tokens in df_filtre['comment_cleaned']:
    word_freq.update(tokens)

# Les mots les plus fréquents
most_common_words = word_freq.most_common(50)
most_common_words

C:\Users\hacho\AppData\Local\Temp\ipykernel_20796\3476422110.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtre['comment_cleaned'] = df_filtre['comment'].apply(preprocess_comment)


[('jeu', 2860),
 ('plus', 1087),
 ('a', 1072),
 ('tout', 752),
 ('très', 743),
 ('bien', 735),
 ('cartes', 695),
 ('joueurs', 588),
 ('si', 563),
 ('bon', 559),
 ('peu', 551),
 ('fait', 470),
 ('partie', 465),
 ('ça', 451),
 ('parties', 435),
 ('peut', 434),
 ('jouer', 431),
 ('jeux', 415),
 ('faire', 397),
 ('trop', 337),
 ('petit', 334),
 ('sans', 326),
 ('faut', 317),
 ('tour', 304),
 ('quand', 297),
 ('comme', 281),
 ('donc', 264),
 ('règles', 263),
 ('temps', 252),
 ('vraiment', 244),
 ('moins', 235),
 ('encore', 235),
 ('simple', 233),
 ('être', 233),
 ('car', 228),
 ('aussi', 226),
 ('non', 216),
 ('assez', 207),
 ('hasard', 202),
 ('beaucoup', 202),
 ('autres', 201),
 ('joueur', 196),
 ('5', 196),
 ('carte', 195),
 ('fois', 192),
 ('là', 192),
 ('tous', 190),
 ('2', 188),
 ('où', 188),
 ('thème', 182)]